In [1]:
import pandas as pd
import numpy as np
import datetime 

In [2]:
df = pd.read_csv("BTCUSDT_historical.csv", index_col="Unnamed: 0", parse_dates=True)

In [132]:
df.head()

,datetime,open,high,low,close,volume
2017-08-17 00:00:00,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181009
2017-08-17 01:00:00,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234916
2017-08-17 02:00:00,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229691
2017-08-17 03:00:00,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443249
2017-08-17 04:00:00,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807


In [133]:
df.tail()

,datetime,open,high,low,close,volume
2022-09-30 22:00:00,1664589600000,19423.06,19436.00,19384.41,19413.79,5773.83532
2022-09-30 23:00:00,1664593200000,19413.96,19438.75,19361.05,19416.05,5861.49212
2022-10-01 00:00:00,1664596800000,19416.05,19423.63,19370.70,19407.43,5958.93517
2022-10-01 01:00:00,1664600400000,19407.43,19418.88,19270.59,19317.92,12420.01718
2022-10-01 02:00:00,1664604000000,19316.82,19324.94,19266.26,19292.82,7000.01196


In [134]:
df.isnull().sum()

datetime    0
open        0
high        0
low         0
close       0
volume      0
dtype: int64

Let's prepare the data and divided trading sessions between day and night
Day: 9 to 13
Night 14 to 8

For this project we will make some naive assumptions

The market is not random
The price of BTC on the previous day and previous night matter for the next day

In [23]:
df_day = df[(df.index.time >= datetime.time(9))
   & (df.index.time <= datetime.time(16))]

In [24]:
df_night = df[(df.index.time <= datetime.time(8)) |
              (df.index.time >= datetime.time(17))]

In [5]:
#df_night = df[~df.index.isin(df_day.index)]

In [137]:
len(df_day), len(df_night), len(df)

(15015, 29985, 45000)

In [25]:
df_day["date"] = pd.to_datetime(df_day.index.date)

C:\Users\Pastor\AppData\Local\Temp\ipykernel_5008\1416156222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_day["date"] = pd.to_datetime(df_day.index.date)


We substract 9 hours from the actual time so night session belong to a single day while maintaining the order

In [ ]:
df_night["date"] = pd.to_datetime(df_night.index.date)

df_night["date"] = df_night.index - pd.to_timedelta(9, unit="h")

In [33]:
df_night.index = df_night.date

In [34]:
ohlc_dict = {
    "open": "first",
    "high": "max",
    "low": "min",
    "close": "last",
    "volume": "sum"}

day_sum = df_day.resample('1d').apply(ohlc_dict).dropna()
night_sum = df_night.resample('1d').apply(ohlc_dict).dropna()

In [142]:
#night_sum = night_sum.shift().dropna()

In [35]:
day_night = day_sum.merge(night_sum, left_index=True, right_index=True, suffixes=('_day', '_night'))

In [41]:
day_night

,open_day,high_day,low_day,close_day,volume_day,open_night,high_night,low_night,close_night,volume_night,phigh_day_touch,ret_distance_phigh_day_open,plow_day_touch,ret_distance_plow_day_open,phigh_night_touch,ret_distance_phigh_night_open,plow_night_touch,ret_distance_plow_night_open
2017-08-17,4354.18,4369.69,4200.74,4286.53,489.856312,4269.36,4371.52,4134.61,4250.34,859.294935,0,NaN,0,NaN,0,NaN,0,NaN
2017-08-18,4250.34,4297.75,3938.77,4163.49,299.466732,4163.49,4184.69,3850.00,4000.00,308.809447,0,-0.027313,1,0.011807,0,-0.027720,1,0.027991
2017-08-19,3985.00,4149.99,3976.72,4086.09,33.535363,4086.09,4211.08,4032.62,4131.46,121.371456,0,-0.072771,0,0.011737,0,-0.047719,0,0.035065
2017-08-20,4131.46,4185.94,4062.78,4108.28,396.458967,4108.28,4114.86,3911.79,4029.05,520.279600,1,-0.004465,0,0.038911,0,-0.018907,0,0.024510
2017-08-21,4019.99,4070.49,3789.95,3790.00,178.711152,3790.00,3990.12,3400.00,3905.84,657.437283,0,-0.039645,1,-0.010532,0,-0.023055,1,0.027660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-02,19201.72,19341.58,18920.35,19091.24,91553.532990,19091.99,19498.00,19059.68,19409.62,198746.550580,0,-0.007285,1,0.000090,0,-0.010012,1,0.008706
2022-10-03,19410.78,19722.60,19407.87,19562.41,90641.762050,19562.41,20270.00,19490.60,19992.69,223959.944270,0,0.003578,0,0.025921,1,-0.004473,0,0.018421
2022-10-04,19992.69,20475.00,19872.02,20181.95,105452.536520,20181.95,20299.99,19730.00,20231.58,208810.880620,0,0.013694,0,0.030133,1,-0.013681,0,0.025761
2022-10-05,20231.58,20389.00,19975.42,20266.82,109152.946810,20266.82,20456.60,19874.81,20052.31,234691.185640,0,-0.011889,0,0.018094,1,-0.003370,0,0.025422


In [37]:
features = ["high_day", "low_day", "high_night", "low_night"]
categorical_features = []
numerical_features = []
for i in features:
    day_night[f"p{i}_touch"] = np.where(
        (day_night[i].shift() <= day_night["high_day"]) & (day_night[i].shift() >= day_night["low_day"]), 1, 0)
        
    day_night[f"ret_distance_p{i}_open"] = (day_night["open_day"] - day_night[i].shift())/day_night[i].shift()
    categorical_features.append(f"p{i}_touch")
    numerical_features.append(f"ret_distance_p{i}_open")

In [40]:
day_night[numerical_features].describe()

,ret_distance_phigh_day_open,ret_distance_plow_day_open,ret_distance_phigh_night_open,ret_distance_plow_night_open
count,1874.000000,1.874000e+03,1874.000000,1874.000000
mean,-0.014754,1.960574e-02,-0.022102,0.027266
std,0.036737,3.955737e-02,0.023309,0.029121
min,-0.237019,-1.978142e-01,-0.233009,0.000001
25%,-0.029766,3.056772e-07,-0.028938,0.009839
50%,-0.011376,1.467852e-02,-0.014145,0.018365
75%,0.003271,3.713143e-02,-0.007164,0.034820
max,0.143042,2.362646e-01,0.000000,0.374033


In [38]:
export_data = day_night[categorical_features + numerical_features]
export_data.to_csv("BTC_feature_data.csv")